# Data Download & Cleaning

The goal of this notebook is to download, filter, and clean news & bar data

# 0. Setup

In [25]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import ast
import pandas as pd
import yaml
from tqdm import tqdm
from dotenv import load_dotenv
from collections import Counter
from alpaca_api import AlpacaRequester

tqdm.pandas()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
"""
.env structure:

APCA_API_KEY_ID=<your_alpaca_api_key_id>
APCA_API_SECRET_KEY=<your_alpaca_api_secret_key>
"""

assert load_dotenv()  # by default loads from .env in current dir

In [27]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

START = str(config['START'])
END = str(config['END'])
PREDICTION_DELTA = config['PREDICTION_DELTA']
OPEN_CUTOFF = config['OPEN_CUTOFF']
CLOSE_CUTOFF = config['CLOSE_CUTOFF']
FWD_WINDOW = config['FWD_WINDOW']
BACK_WINDOW = config['BACK_WINDOW']

In [28]:
AR = AlpacaRequester()


# 1. Get Data

## 1.1. Getting News Article Headlines

We'll download all news articles from January 1, 2020 to the present

In [29]:
# This will take 2hrs + to run
GET_NEWS = False
if GET_NEWS:
    AR.get_news(start=START,exclude_contentless=False,include_content=False)

Next we'll filter to the symbols we care about, and clean them up. The ignored symbols are ignored because they're:
- not American (we are using American news articles, so best to stick with American stocks)
- too volatile
- too small
- meme stocks
- crypto or crypto-related

In [30]:
CRYPTO_SYMBOLS = ['BTC', 'BTCUSD', 'ETHUSD', 'DOGEUSD', 'SHIBUSD', 'SOLUSD','BTC','$BTC']
ETF_SYMBOLS = ['EWU','FXI','GLD','QQQ','RSX','SPY','USO','VGK','UNG']
BAD_SYMBOLS = ['BBBY','TWTR','ABC','ACB','CGC','TLRY','AMC','GME','NKLA','CVNA','LCID','NVAX','MARA','RIOT','MSTR','PLUG','PTON','SNAP','SPCE','COIN','HOOD','XPEV','NIO','LI','RIVN','RBLX','DIA','ATVI','LYFT','CCL','BABA','BIDU','JD','PDD','SONY','STLA','AZN','NVO','TCEHY','SNDL','TM','GSK','AFRM','DKNG','PENN','BNTX','BBY','BYND','SOFI','SNY']
REPLACE_SYMBOLS = {'FB': 'META','GOOG':'GOOGL'}
def format_symbols(symbols):
    symbols = ast.literal_eval(symbols)
    symbols = [(i if i not in REPLACE_SYMBOLS else REPLACE_SYMBOLS[i]) for i in symbols if i not in CRYPTO_SYMBOLS + ETF_SYMBOLS + BAD_SYMBOLS]
    symbols = set(map(lambda x:x.replace('$','').replace('-','.'),symbols))
    return list(symbols)

articles_df: pd.DataFrame = pd.read_csv("news/news.csv")
articles_df['symbols'] = articles_df['symbols'].progress_apply(format_symbols)

/var/folders/ck/f9wf3kz948538_qhqyrxj0xc0000gn/T/ipykernel_24667/626317480.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  articles_df: pd.DataFrame = pd.read_csv("news/news.csv")
100%|██████████| 1278601/1278601 [00:08<00:00, 142597.18it/s]


In [31]:
symbol_counter = Counter([i for sublist in articles_df['symbols'] for i in sublist])
symbols, _ = zip(*symbol_counter.most_common(100))
symbols = set(symbols)
symbols

{'AAL',
 'AAPL',
 'ABBV',
 'ABNB',
 'ABT',
 'ACN',
 'ADBE',
 'AMAT',
 'AMD',
 'AMGN',
 'AMZN',
 'AVGO',
 'AXP',
 'BA',
 'BAC',
 'BB',
 'BIIB',
 'BLK',
 'BMY',
 'BX',
 'C',
 'CHWY',
 'CMCSA',
 'CMG',
 'COST',
 'CRM',
 'CRWD',
 'CSCO',
 'CVS',
 'CVX',
 'DAL',
 'DASH',
 'DELL',
 'DIS',
 'ENPH',
 'F',
 'FDX',
 'GE',
 'GILD',
 'GM',
 'GOOGL',
 'GS',
 'HD',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'LLY',
 'LMT',
 'LULU',
 'LUV',
 'MA',
 'MCD',
 'META',
 'MGM',
 'MRK',
 'MRNA',
 'MRVL',
 'MS',
 'MSFT',
 'MU',
 'NFLX',
 'NKE',
 'NOW',
 'NVDA',
 'ORCL',
 'OXY',
 'PANW',
 'PFE',
 'PINS',
 'PLTR',
 'PYPL',
 'QCOM',
 'RCL',
 'REGN',
 'ROKU',
 'SBUX',
 'SHOP',
 'SMCI',
 'SNOW',
 'SPOT',
 'SQ',
 'T',
 'TGT',
 'TMUS',
 'TSLA',
 'TSM',
 'UAL',
 'UBER',
 'UNH',
 'UPS',
 'V',
 'VZ',
 'WBD',
 'WFC',
 'WMT',
 'XOM',
 'ZM',
 'ZS'}

In [32]:
# As of mid July 2025, this will download about 60GB+ of data
GET_BARS = False
if GET_BARS:
    AR.get_bars(store_url=False,store_token=False,symbols=symbols,start=START,timeframe="1T",adjustment="all")

# 2. Data cleaning & filtering

## 2.1. Clean & store bars

We will restrict to only those bars that:
- Occur during normal trading hours, according to the calendar
- Occur on days where that stock has at least 60 1-minute trading bars during normal trading hours

The reason for this is because there are some mistakes in the Alpaca dataset, for example, some days only have one bar for each ticker.

We also specifically get rid of any bars for a ticker for 2022-01-24 if there are 75 or less bars for that day, as there seems to be an error with that day where for some tickers only the first 75 minutes of bars are available.

Lastly, we get rid of any IPO days, as they are much too volatile

In [ ]:
calendar = AR.market_calendar(start=START,end=END)[['date','open','close']]
calendar["open"] = pd.to_datetime(
    calendar["date"].dt.strftime("%Y-%m-%d") + " " + calendar["open"],
    format="%Y-%m-%d %H:%M",
).dt.tz_localize("America/New_York")
calendar['close'] = pd.to_datetime(
    calendar['date'].dt.strftime('%Y-%m-%d') + " " + calendar['close'],
    format='%Y-%m-%d %H:%M'
).dt.tz_localize('America/New_York')
calendar.rename(columns={'open':'mkt_open','close':'mkt_close'},inplace=True)

In [34]:
from pathlib import Path

# Dictionary storing dates with bars after filtering for each ticker
# These dates will be used to filter articles later
dates = {}

IPOs = {
    'ABNB': pd.Timestamp(2020,12,10),
    'DASH': pd.Timestamp(2020,12,9),
    'PLTR': pd.Timestamp(2020,9,30),
    'SNOW': pd.Timestamp(2020,9,16),
}

for symbol in tqdm(symbols,smoothing=0):
    path = Path(f'bars/{symbol}.csv')
    if path.exists() and path.is_file():
        df = pd.read_csv(f'bars/{symbol}.csv')
        df.drop_duplicates('t',keep='first',inplace=True)

        # Format time and make date column
        df['t'] = pd.to_datetime(df['t']).dt.tz_convert('America/New_York')
        df['date'] = df['t'].dt.tz_convert(None).dt.normalize()

        # Get mkt open and close times
        df = df.merge(calendar[['date','mkt_open','mkt_close']],on='date',how='left')

        # Filter to only bars that occur during the trading day
        df = df[df['t'].between(df['mkt_open'],df['mkt_close'],inclusive='left')]

        # Create dates that we'll keep; we want to get rid of:
        # - IPO days
        # - The aforementioned error day on 2022-01-24
        # - days with < 60 bars
        def filter_days(day_bars): 
            date = day_bars.name
            c1 = not (symbol in IPOs and date == IPOs[symbol]) 
            c2 = (date != pd.Timestamp(2022,1,24) or len(day_bars) > 75) 
            c3 = (len(day_bars) >= 60)
            return c1 and c2 and c3
        dates[symbol] = set(df.groupby('date').filter(filter_days)['date'])

        # Write to parquet file
        df.to_parquet(f'bars/{symbol}.parquet',index=False)
    else:
        print("Missing bars for {tkr}")

100%|██████████| 100/100 [01:45<00:00,  1.05s/it]


## 2.2. Clean & store articles

### 1. Select & format column types

First, we'll format the columns to the proper timezone and with the right types; this will make the dataframe take up less memory

In [35]:
# Get only cols we care about
articles_df = articles_df[['created_at','updated_at','headline','symbols']]  # type: ignore
articles_df: pd.DataFrame

articles_df.dtypes

created_at    object
updated_at    object
headline      object
symbols       object
dtype: object

In [36]:
articles_df['headline'] = articles_df['headline'].astype("string")
articles_df['created_at'] = pd.to_datetime(articles_df['created_at']).dt.tz_convert("America/New_York")
articles_df['updated_at'] = pd.to_datetime(articles_df['updated_at']).dt.tz_convert("America/New_York")
articles_df.dtypes

created_at    datetime64[ns, America/New_York]
updated_at    datetime64[ns, America/New_York]
headline                        string[python]
symbols                                 object
dtype: object

### 2. Filter to universe

We only want articles with tickers in our list of symbols

In [37]:
symbols = set(symbols)

# Only keep articles that have at least one symbol in our list
articles_df['symbols'] = articles_df['symbols'].apply(lambda x: [i for i in x if i in symbols])
articles_df = articles_df[articles_df['symbols'].apply(len) > 0]  # type: ignore

### 3. Get rid of articles updated more than a minute after creation date

We want to get rid of articles which were updated more than a minute after they were created to avoid lookahead bias. To start, we'll see how many articles we'd be getting rid of:

In [38]:
mask = (articles_df['updated_at'] - articles_df['created_at']).dt.total_seconds() <= 60
print(f"{100 * len(articles_df[~mask]) / len(articles_df):.2f}% of articles have been updated after 60 seconds")
# Outputs: 5.52% of articles have been updated after 60 seconds

5.52% of articles have been updated after 60 seconds


We're retaining 94.48% of articles, so this shouldn't impact coverage too badly

In [39]:
articles_df = articles_df[mask]

### 4. Get rid of headlines which appear more than once and more than an hour apart

Next we'll get rid of any rows whose headline appears more than once with at least one occurrence being more than one hour apart. Any such headline is inherently not time-senstive, so we don't care about it.

In [40]:
g = articles_df.groupby('headline')['created_at']
mask = (g.transform('max') - g.transform('min')) <= pd.Timedelta('1h')
articles_df = articles_df[mask]

### 5. Keep first occurrence of each headline

Now, we'll keep the first occurrence of each headline (if there are multiple headlines in a single hour presumably it is a duplicate article; we'll keep the first)

In [41]:
articles_df = articles_df.sort_values('created_at').drop_duplicates(subset='headline', keep='first')

### 6. Keep only articles which ocurred during trading hours for days we have bars

In [42]:
# Make date column
articles_df['date'] = articles_df['created_at'].dt.tz_localize(None).dt.normalize()  # type: ignore

# Make dataframe of trading dates by ticker
date_df = pd.DataFrame([{'symbol': sym, 'date': d} for sym,ds in dates.items() for d in ds])

In [43]:
# Explode to one symbol per row and filter to articles which occur on dates for which we have bars
articles_df = articles_df.explode('symbols').rename(columns={'symbols':'symbol'}).merge(date_df, on=['symbol', 'date'])
articles_df['symbol'] = articles_df['symbol'].astype('string')

In [44]:
# Get mkt open and close times
articles_df = articles_df.merge(calendar,on='date',how='left')

In [45]:
# Filter to articles which occur during trading hours, and within the specified cutoff range
mask = (
    (~articles_df['mkt_open'].isna()) &
    (~articles_df['mkt_close'].isna()) &
    (articles_df['created_at'] >= articles_df['mkt_open'] + pd.Timedelta(minutes=OPEN_CUTOFF)) &
    (articles_df['created_at'] < articles_df['mkt_close'] - pd.Timedelta(minutes=CLOSE_CUTOFF + 1))
)
articles_df = articles_df[mask].drop(columns=['date','mkt_open','mkt_close'])

### 7. Write to file

In [46]:
articles_df.to_parquet('news/cleaned_news.parquet')

## 2.3. Store symbols

In [47]:
with open('symbols.pkl', 'wb') as f:
    pickle.dump(symbols,f)


## 2.4. Store dates

In [48]:
date_df.to_parquet('news/valid_dates.parquet')